# **Classification de ressentis avec CamemBERT**

L'objectif est de créer un modèle qui prend en entrée des commentaires (en Français) et attribue à chacun un ressenti positif ou négatif.  
Le modèle global est composé de deux parties :  
* [CamemBERT](https://camembert-model.fr/) va encoder le commentaire et en extraire des informations qui seront passées ensuite au réseau de neurones.  
* Le modèle suivant est un réseau de neurones qui sera créé avec l'API [Keras](https://www.tensorflow.org/guide/keras?hl=fr) de [Tensorflow](https://www.tensorflow.org/?hl=fr).  

<img src="https://raw.githubusercontent.com/AlexandreBourrieau/ML-F1/master/Carnets%20Jupyter/Images/StructureBERT.png" />  
  
  Les données qui s'échangent entre les deux modèles sont des vecteurs de dimension 768. On peut voir ces vecteurs comme l'équivalent de l'application d'un algorithme de prolongation lexicale sur les mots qui composent le commentaire.

In [2]:
!pip install transformers --quiet

     |████████████████████████████████| 1.1MB 9.6MB/s 
     |████████████████████████████████| 3.0MB 32.9MB/s 
     |████████████████████████████████| 1.1MB 53.7MB/s 
     |████████████████████████████████| 890kB 53.8MB/s 


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, Input, Dropout, Lambda
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

from transformers import CamembertConfig
from transformers import TFCamembertModel
from transformers import AutoTokenizer

import matplotlib.pyplot as plt

# Importation des données

Téléchargement des ressentis Allociné

In [4]:
# Téléchargement des données depuis le repot github "https://github.com/AlexandreBourrieau/ML-F1/raw/master/Carnets%20Jupyter/Donn%C3%A9es/data.tar.bz2"

!wget -q "https://github.com/AlexandreBourrieau/ML-F1/raw/master/Carnets%20Jupyter/Donn%C3%A9es/data.tar.bz2"
!tar -xjvf data.tar.bz2

data/
data/allocine_dataset.pickle
data/test.jsonl
data/train.jsonl
data/val.jsonl


In [5]:
df = pd.read_json("/content/data/test.jsonl", lines=True)
df.head(5)

,film-url,review,polarity
0,http://www.allocine.fr/film/fichefilm-25385/cr...,"Magnifique épopée, une belle histoire, touchan...",1
1,http://www.allocine.fr/film/fichefilm-1954/cri...,Je n'ai pas aimé mais pourtant je lui mets 2 é...,0
2,http://www.allocine.fr/film/fichefilm-135523/c...,Un dessin animé qui brille par sa féerie et se...,1
3,http://www.allocine.fr/film/fichefilm-61514/cr...,"Si c'est là le renouveau du cinéma français, c...",0
4,http://www.allocine.fr/film/fichefilm-260395/c...,Et pourtant on s’en Doutait !Second volet très...,0


Affiche quelques informations :

In [6]:
def LongueurMax(df):
  Lmax = 0
  for com in df['review']:
    Longueur = len(com)
    if Lmax < Longueur:
      Lmax = Longueur
  return Lmax

In [7]:
print(df[0:10])
print("Total des données : ", str(len(df)))
print("Nombre d'avis positifs et négatifs : ",df['polarity'].value_counts())
print("Longueur maximale d'un commentaire : ",LongueurMax(df))

                                            film-url  ... polarity
0  http://www.allocine.fr/film/fichefilm-25385/cr...  ...        1
1  http://www.allocine.fr/film/fichefilm-1954/cri...  ...        0
2  http://www.allocine.fr/film/fichefilm-135523/c...  ...        1
3  http://www.allocine.fr/film/fichefilm-61514/cr...  ...        0
4  http://www.allocine.fr/film/fichefilm-260395/c...  ...        0
5  http://www.allocine.fr/film/fichefilm-220641/c...  ...        1
6  http://www.allocine.fr/film/fichefilm-120103/c...  ...        1
7  http://www.allocine.fr/film/fichefilm-190956/c...  ...        1
8  http://www.allocine.fr/film/fichefilm-186185/c...  ...        1
9  http://www.allocine.fr/film/fichefilm-17327/cr...  ...        0

[10 rows x 3 columns]
Total des données :  20000
Nombre d'avis positifs et négatifs :  0    10408
1     9592
Name: polarity, dtype: int64
Longueur maximale d'un commentaire :  2000


# **Préparation des données**


In [8]:
MAX = 3000

# Chargement des commentaires et des ressentis
commentaires = df['review'].astype(str).tolist()    # Récupère tous les commentaires dans une liste python
ressentis = df['polarity'].tolist()                   # Récupère tous les ressentis dans une liste python
labels = np.asarray(ressentis)               # Créé un tableau de type numpy avec les ressentis

x_entrainement, x_test, y_entrainement, y_test = train_test_split(commentaires[0:MAX], labels[0:MAX], test_size=0.25)

In [8]:
print ("Nombre de commentaires pour l'entrainement : ", len(x_entrainement))
print ("Nombre de commentaires pour les tests : ", len(x_test))

Nombre de commentaires pour l'entrainement :  2250
Nombre de commentaires pour les tests :  750


# **Tokénisation**


In [9]:
LONGUEUR_MAX_COMMENTAIRE = LongueurMax(df) + 2

# Instanciation du tokeniseur
tokenizer = AutoTokenizer.from_pretrained('jplu/tf-camembert-base')

# Préparation des données d'entrainement
output_tokenizer_entrainement = tokenizer(x_entrainement,max_length=LONGUEUR_MAX_COMMENTAIRE, padding='max_length', truncation=False, return_tensors='tf',add_special_tokens=True)

# Préparation des données de tests
output_tokenizer_tests = tokenizer(x_test,max_length=LONGUEUR_MAX_COMMENTAIRE, padding='max_length', truncation=False, return_tensors='tf',add_special_tokens=True)

Regardons un peu comment sont formatées les données en sortie du tokéniseur :

In [10]:
output_tokenizer_entrainement

{'input_ids': <tf.Tensor: shape=(2250, 2002), dtype=int32, numpy=
array([[    5,   100,    49, ...,     1,     1,     1],
       [    5,    54,  2370, ...,     1,     1,     1],
       [    5,   153,   918, ...,     1,     1,     1],
       ...,
       [    5, 11146,   492, ...,     1,     1,     1],
       [    5, 26965,    15, ...,     1,     1,     1],
       [    5,   148,   492, ...,     1,     1,     1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2250, 2002), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

Regardons comment le premier commentaire a été encodé :

In [30]:
print("Commentaire original :", x_entrainement[0])
print("input_ids: ", output_tokenizer_entrainement['input_ids'][0])
print("attention_mask: ", output_tokenizer_entrainement['attention_mask'][0])

Commentaire original : Très bon film. On se prend facilement dans cette histoire qui en fait en est deux. On s'y prend très vite au jeu et on regarde le développement de chacune. Deux films en un! C est un film précieux.
input_ids:  tf.Tensor([   5 1508  212 ...    1    1    1], shape=(2002,), dtype=int32)
attention_mask:  tf.Tensor([1 1 1 ... 0 0 0], shape=(2002,), dtype=int32)


# **Définition et utilisation du modèle camemBERT avec Keras**

In [13]:
# Instanciation du modèle camemBERT
transformer_model = TFCamembertModel.from_pretrained('jplu/tf-camembert-base')

# Défintion du format des entrées du modèle
entrees_ids = tf.keras.layers.Input(shape=(LONGUEUR_MAX_COMMENTAIRE,), name='input_token', dtype='int32')
entrees_masks = tf.keras.layers.Input(shape=(LONGUEUR_MAX_COMMENTAIRE,), name='masked_token', dtype='int32') 

# Création de la sortie du modèle
sortie_camemBERT = transformer_model([entrees_ids,entrees_masks])

# Instanciation du modèle avec Keras
model_camemBERT = tf.keras.Model(inputs=[entrees_ids, entrees_masks], outputs = sortie_camemBERT,trainable=False)
model_camemBERT.summary()

Some weights of the model checkpoint at jplu/tf-camembert-base were not used when initializing TFCamembertModel: ['lm_head']
- This IS expected if you are initializing TFCamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFCamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFCamembertModel were initialized from the model checkpoint at jplu/tf-camembert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 2002)]       0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 2002)]       0                                            
__________________________________________________________________________________________________
tf_camembert_model_2 (TFCamembe ((None, 2002, 768),  110621952   input_token[0][0]                
                                                                 masked_token[0][0]               
Total params: 110,621,952
Trainable params: 0
Non-trainable params: 110,621,952
__________________________________________________________________________________________________


Si on regarde le format de la sortie du modèle camemBERT, on voit qu'elle est composée de deux sorties :
* Une sortie avec un format (None,MAX_SEQUENCE_LENGTH,768)
* Une sortie avec un format (None,768)  
  
On trouve la signification de ces sorties [sur le site de hugginface](https://huggingface.co/transformers/main_classes/output.html#tfbasemodeloutput). Ainsi, la première sortie est de type `last_hidden_state` et la deuxième de type `pooler_output`.

In [15]:
sortie_camemBERT

(<tf.Tensor 'tf_camembert_model_2/roberta/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0' shape=(None, 2002, 768) dtype=float32>,
 <tf.Tensor 'tf_camembert_model_2/roberta/pooler/dense/Tanh:0' shape=(None, 768) dtype=float32>)

 Celle qui nous interesse ici est la sortie `last_hidden_state` : C'est elle qui contient le résultat de l'encodage des mots des commentaires

In [16]:
sortie_camemBERT[0]

<tf.Tensor 'tf_camembert_model_2/roberta/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0' shape=(None, 2002, 768) dtype=float32>